# Recommender Systems with DGL

## Introduction

Graph Neural Networks (GNN), as a methodology of learning representations on graphs, has gained much attention recently.  Various models such as Graph Convolutional Networks, GraphSAGE, etc. are proposed to obtain representations of whole graphs, or nodes on a single graph.

A primary goal of Collaborative Filtering (CF) is to automatically make predictions about a user's interest, e.g. whether/how a user would interact with a set of items, given the interaction history of the user herself, as well as the histories of other users.  The user-item interaction can also be viewed as a bipartite graph, where users and items form two sets of nodes, and edges connecting them stands for interactions.  The problem can then be formulated as a *link-prediction* problem, where we try to predict whether an edge (of a given type) exists between two nodes.

Based on this intuition, the academia developed multiple new models for CF, including but not limited to:

* Geometric Learning Approaches
  * [Geometric Matrix Completion](https://papers.nips.cc/paper/5938-collaborative-filtering-with-graph-information-consistency-and-scalable-methods.pdf)
  * [Recurrent Multi-graph CNN](https://arxiv.org/pdf/1704.06803.pdf)
* Graph-convolutional Approaches
  * Models such as [R-GCN](https://arxiv.org/pdf/1703.06103.pdf) or [GraphSAGE](https://github.com/stellargraph/stellargraph/tree/develop/demos/link-prediction/hinsage) also apply.
  * [Graph Convolutional Matrix Completion](https://arxiv.org/abs/1706.02263)
  * [PinSage](https://arxiv.org/pdf/1806.01973.pdf)

## Dependencies

* Latest DGL build: `conda install -c dglteam dgl`
* `pandas`
* `stanfordnlp`
* `pytorch`

## Loading data

In this tutorial, we focus on rating prediction on MovieLens-1M dataset.

After loading and train-validation-test-splitting the dataset, we process the movie title into word count vectors, and other features into categorical variables.  We then store them as node features on the graph.

Since user features and item features are different, we pad both types of features with zeros.

In [ ]:
import torch.nn as nn

In [ ]:
class MovieLens(object):
    pass

## Model

We can now write a GraphSAGE layer.  In GraphSAGE, the node representation is updated with the representation in the previous layer as well as an aggregation (often mean) of "messages" sent from all neighboring nodes.

In [ ]:
class GraphSageConv(nn.Module):
    pass

## Sampling

Ideally, we wish to execute a full update of the node embeddings with the GraphSAGE layer.  However, when the graph scales up, the full update soon becomes impractical, because the node embeddings couldn't fit in the GPU memory.

A natural solution would be partitioning the nodes and computing the embeddings one partition (minibatch) at a time.  The nodes at one convolution layer then only depends on their neighbors, rather than all the nodes in the graph, hence reducing the computational cost.  However, if we have multiple layers, and some of the nodes have a lot of neighbors (which is often the case since the degree distribution of many real-world graphs follow [power-law](https://en.wikipedia.org/wiki/Scale-free_network)), then the computation may still eventually depend on every node in the graph.

*Neighbor sampling* is an answer to further reduce the cost of computing node embeddings.  When aggregating messages, instead of collecting from all neighboring nodes, we only collect from some of the randomly-sampled (for instance, uniform sampling at most K neighbors without replacement) neighbors.

DGL provides the `NodeFlow` object that describes the computation dependency of nodes in a graph convolutional network, as well as various samplers that constructs such `NodeFlow`s as graphs.  From a programmer's perspective, training with minibatch and neighbor sampling reduces to propagating the messages in `NodeFlow` as follows.

In [ ]:
sampler = NeighborSampler()


## Training

Training now only involves iterating over the neighbor sampler, propagating the messages, and computing losses and gradients as usual.

Meanwhile, we also evaluate the RMSE on validation and test set.